In [55]:
import os
import pandas as pd
import lightgbm as lgbm
from sklearn.model_selection import train_test_split

# Model framework

Simple model build

## Load datasets

In [56]:
dataset_folder = os.path.join(os.pardir, 'Datasets')
training_feature_files = [
    'FEATURE_entity_counts_train.csv',
    'FEATURE_topic_sharing_train.csv'
]

In [57]:
X_train = pd.read_csv(os.path.join(dataset_folder, 'train.csv'), index_col='id', nrows=5000)
for feature_set in training_feature_files:
    feature_df = pd.read_csv(os.path.join(dataset_folder, feature_set), index_col='id')
    X_train = X_train.merge(feature_df, left_index=True, right_index=True)
y_train = X_train.pop('is_duplicate')

In [58]:
metadata = ['qid1', 'qid2', 'question1', 'question2']
X_train.drop(columns=metadata, inplace=True)

In [59]:
print(X_train.shape)
X_train.head()

(5000, 28)


PERSON_COUNT  NORP_COUNT  FAC_COUNT  ORG_COUNT  GPE_COUNT  LOC_COUNT  \
id                                                                         
0              0           0          0          0          1          0   
1              0           0          0          0          0          0   
2              0           0          0          0          0          0   
3              0           0          0          0          0          0   
4              0           0          0          0          0          0   

    PRODUCT_COUNT  EVENT_COUNT  WORK_OF_ART_COUNT  LAW_COUNT  \
id                                                             
0               0            0                  0          0   
1               0            0                  0          0   
2               0            0                  0          0   
3               0            0                  0          0   
4               0            0                  0          0   

             ...             PROB_BOTH_SHARE_TOPIC_0  PROB_BOTH_SHARE_TOPIC_1  \
id           ...                                                                
0            ...                            0.000179                 0.000179   
1            ...                            0.000556                 0.000556   
2            ...                            0.000286                 0.000286   
3            ...                            0.000500                 0.020500   
4            ...                            0.000250                 0.030249   

    PROB_BOTH_SHARE_TOPIC_2  PROB_BOTH_SHARE_TOPIC_3  PROB_BOTH_SHARE_TOPIC_4  \
id                                                                              
0                  0.000179                 0.000179                 0.000179   
1                  0.000556                 0.000556                 0.011667   
2                  0.034571                 0.000286                 0.000286   
3                  0.000500                 0.000500                 0.000500   
4                  0.000250                 0.395254                 0.000250   

    PROB_BOTH_SHARE_TOPIC_5  PROB_BOTH_SHARE_TOPIC_6  PROB_BOTH_SHARE_TOPIC_7  \
id                                                                              
0                  0.001964                 0.664464                 0.000179   
1                  0.000556                 0.128333                 0.006111   
2                  0.305993                 0.000286                 0.000286   
3                  0.000500                 0.015500                 0.000500   
4                  0.000250                 0.000250                 0.002750   

    PROB_BOTH_SHARE_TOPIC_8  PROB_BOTH_SHARE_TOPIC_9  
id                                                    
0                  0.000179                 0.000179  
1                  0.000556                 0.006111  
2                  0.000286                 0.003143  
3                  0.000500                 0.000500  
4                  0.000250                 0.000250  

[5 rows x 28 columns]

In [60]:
y_train.shape

(5000,)

## Train model

In [61]:
X_train, X_eval, y_train, y_eval = train_test_split(X_train, y_train, test_size=.2, random_state=0)

In [69]:
lgbm_train = lgbm.Dataset(
    X_train, 
    label=y_train,
    free_raw_data=False
)

lgbm_eval = lgbm.Dataset(
    X_eval, 
    label=y_eval,
    free_raw_data=False
)

lgbm_params = {
    'application': 'binary',
    'min_data_in_leaf': 40,
    'boosting_type ': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'metric': 'logloss'
}

In [73]:
evals_result = {}

In [74]:
clf = lgbm.train(params=lgbm_params, 
                 train_set=lgbm_train, 
                 valid_sets=[lgbm_train, lgbm_eval],
                 valid_names=['train', 'eval'],
                 verbose_eval=10,
                 early_stopping_rounds=20,
                 num_boost_round=200,
                 evals_result=evals_result
                 )

ValueError: For early stopping, at least one dataset and eval metric is required for evaluation